In [ ]:
from johnson import *

In [ ]:
# corpus = Corpus('ecco_tcp')
# corpus2 = Corpus('tedjdh')
# meta = pd.concat([corpus2.meta, corpus.meta]).reset_index().drop_duplicates('id',keep='first').set_index('id')
corpus = Corpus('tedjdh')
meta = corpus.meta
meta['num_words']=[len(' '.join(corpus.textd[idx].sents()[:1000]).split()) for idx in tqdm(meta.index)]

In [ ]:
# meta[['num_words2','num_words']]
goodmeta=meta.query('ocr_accuracy>=.9')[['author','medium','num_words','title','year','ocr_accuracy']]
goodmeta

In [ ]:
def unpack_parallel(xyz):
    x,y,z = xyz
    words = (tuple(a[0] for a in x), y[0], tuple(b[0] for b in z))
    pos = (tuple(a[1] for a in x), y[1], tuple(b[1] for b in z))
    words_flat = tuple(list(words[0]) + [words[1]] + list(words[2]))
    is_valid = not all(w == w.upper() for w in words_flat)
    plen = len(x)
    midword = words[1]#.lower()
    # if midword=='&': midword='and'
    return {
        'word_beg': ' '.join(words[0]),
        'word_mid': midword,
        'word_end': ' '.join(words[2]),

        'pos_beg':' '.join(pos[0]),
        'pos_mid':pos[1],
        'pos_end':' '.join(pos[2]),

        'plen':plen,
        'is_valid':is_valid,
    }

def get_db_data(suffix=''):
    db=get_db(flag='r',suffix=suffix)
    o=[]
    for key in tqdm(db.keys(), total=len(db)):
        for sent_num, sent_parallels in db[key]:
            for paral in sent_parallels:
                pdat = unpack_parallel(paral)
                if pdat['is_valid']:
                    o.append({'id':key, 'sent_num':sent_num, **pdat})
    return pd.DataFrame(o).set_index('id')
        

In [ ]:
data = get_db_data(suffix='')
data

In [ ]:
datameta=data.join(goodmeta,how='inner')
datameta

In [ ]:
datameta.plen.value_counts()

In [ ]:
datameta[datameta.plen==5]

In [ ]:
datameta[datameta.author.str.contains('Johnson, Samuel')].query('plen>2')

In [ ]:
datameta.query('plen==3').author.value_counts()

In [183]:
norms=get_allnorms()['Abs-Conc.Median.C18'].dropna()
norms=norms.loc[norms.index.dropna()]
norms_d = dict(norms)

In [ ]:
data[data.is_valid].query('plen>1').word_mid.value_counts().head(25)

In [ ]:

# meta=corpus.meta.join(corpus2.meta, how='outer', lsuffix='', rsuffix='_tedjdh')
# meta=pd.concat([meta, corpus3.meta.query('1700>year<1900').assign(medium='Fiction')])

In [ ]:
datameta=data.set_index('id').join(meta, lsuffix='_data', rsuffix='_meta')
datameta

In [ ]:
k='num_parallels_per_1000_words'
datameta.groupby('medium').mean(numeric_only=True).sort_values(k)

In [ ]:
top=datameta.query('num_words_data>=5000').groupby('author').mean(numeric_only=True).sort_values(k,ascending=False)
top.head(20)

In [ ]:
import plotnine as p9
k='num_parallels_per_1000_words'
datameta['year']=pd.to_numeric(datameta['year'],errors='coerce')
figdf=datameta.query('1700<=year<1900 & num_words_data>=5000')
figdf=figdf[figdf.medium.isin({'Oratory','Biography','Non-Fiction','Poetry','Drama','Fiction'})]
fig=p9.ggplot(figdf,p9.aes(x='year', y=k, color='medium'))
fig+=p9.geom_point(size=.1)
fig+=p9.geom_smooth(se=True,method='loess')
fig+=p9.facet_wrap('medium')
fig

In [ ]:
datameta[datameta.author.str.contains('Johnson, Samuel')][['medium','title']]